In [1]:
# Import required packages
import os
import mintpy

In [ ]:
orbit = 'DT56'
year = '2017'
frame = 'frame_3'
data_path = f'../proc/data/signal_mintpy/{orbit}_crop/{frame}'

In [ ]:
os.mkdir(f'{data_path}/mintpy_{year}')

In [4]:
# Function to write to MintPy config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [8]:
CONFIG_TXT = f'''# vim: set filetype=cfg:
##----------------------------- hyp3 ---------------------##
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {data_path}/*/*{year}*unw_phase.tif
mintpy.load.corFile          = {data_path}/*/*{year}*corr.tif
##---------geometry datasets:
mintpy.load.demFile          = {data_path}/*/*{year}*dem.tif
mintpy.load.incAngleFile     = {data_path}/*/*{year}*lv_theta.tif
mintpy.load.waterMaskFile    = {data_path}/*/*{year}*water_mask.tif

mintpy.deramp                = linear
mintpy.reference.lalo        = auto
mintpy.troposphericDelay.method   = height_correlation
mintpy.troposphericDelay.minCorrelation = 0.8
'''

config_file = os.path.join(f'{data_path}/mintpy_{year}', f"{frame}_{year}_Sen{orbit}.txt")
write_config_file(config_file, CONFIG_TXT, mode='w')

write configuration to file: /mnt/c/Users/qbren/Desktop/taco/projects/uw_subsidence/proc/mintpy/uw_subsidenceSenAT64.txt


In [ ]:
def run_mintpy(config_file):
    !smallbaselineApp.py f'{config_file}' --dostep load_data
    !smallbaselineApp.py f'{config_file}' --dostep modify_network
    !smallbaselineApp.py f'{config_file}' --dostep reference_point
    !smallbaselineApp.py f'{config_file}' --dostep quick_overview
    !smallbaselineApp.py f'{config_file}' --dostep invert_network
    !smallbaselineApp.py f'{config_file}' --dostep correct_troposphere
    !smallbaselineApp.py f'{config_file}' --dostep deramp
    !smallbaselineApp.py f'{config_file}' --dostep correct_topography
    !smallbaselineApp.py f'{config_file}' --dostep velocity

In [ ]:
run_mintpy(config_file)